In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import json
import Functions
import us

from dash import Dash, html, dcc, Input, Output, callback

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

## Save data
track = {"clicks" : 1}

## Global Figure
fig = Functions.plot_global_country()

fig.update_layout(clickmode='event+select')

og_fig = fig

app.layout = html.Div([
    html.Div([
        dcc.Graph(
            id='map',
            figure=fig
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div(id='track-data', style={'padding': '20px', 'border': '1px solid black', 'width': '49%', 'display': 'inline-block'})
])

@callback(
    Output('map', 'figure'),
    Output('track-data', 'children'),
    [Input('map', 'clickData'),
    Input('map', 'figure')])
def clickData_Change(clickData, figure):

    print("Triggered")
    
    if clickData is not None:
        print(clickData["points"])#[0]["hovertext"])
        val = clickData["points"][0]["location"]
            

        if track["clicks"] == 1: # 2 means chosen once

            print("THIS")
            track["clicks"] += 1

            ## Save selected country
            track["Country"] = clickData["points"][0]["location"]

            ## Option 1 is US States
            if val == "USA":

                fig = Functions.plot_usa_states()
                
                ## Placeholder for states to fill value in later
                track["State"] = ""

            ## Option 2 is NUTS Countries
            elif val in Functions.nuts_countries:

                fig = Functions.plot_nuts_country(val, 1)

                ## Placeholder for nuts 1 to fill values in later
                track["NUTS1"] = ""

                print(clickData["points"])

            ## Option 3 return Figure as is or return the Sectors
            else:

                fig = go.Figure(figure)

        elif track["clicks"] == 2: # 3 means 3 selections

            print("THIS2")
            track["clicks"] += 1

            ## Option 1 is US States
            if track["Country"] == "USA":

                ## Save selected state
                track["State"] = clickData["points"][0]["location"]

                fig = Functions.plot_usa_subnational(track["State"])
                
                ## Placeholder for states to fill value in later
                track["County"] = ""

            ## Option 2 is NUTS Countries
            elif track["Country"] in Functions.nuts_countries:

                ## Save selected NUTS1
                track["NUTS1"] = clickData["points"][0]["location"]

                fig = Functions.plot_nuts_country(track["Country"], 2, track["NUTS1"])

                ## Placeholder for nuts 2 to fill values in later
                track["NUTS2"] = ""

                print(clickData["points"])

            ## Option 3 return Figure as is or return the Sectors
            else:

                fig = go.Figure(figure)

        elif track["clicks"] == 3: # 3 means 3 selections

            print("THIS3")
            track["clicks"] += 1

            ## Option 1 is US States
            if track["Country"] == "USA":

                fig = fig = go.Figure(figure)   

            ## Option 2 is NUTS Countries
            elif track["Country"] in Functions.nuts_countries:

                ## Save selected NUTS1
                track["NUTS2"] = clickData["points"][0]["location"]

                fig = Functions.plot_nuts_country(track["Country"], 3, track["NUTS1"], track["NUTS2"])

                ## Placeholder for nuts 3 to fill values in later
                track["NUTS3"] = ""

                print(clickData["points"])

            ## Option 3 return Figure as is or return the Sectors
            else:

                fig = go.Figure(figure)            

        # if val in Functions.nuts_countries and track.get("nuts1") is None:

        #     print("This")

        #     fig = Functions.plot_nuts_country(val, 1)

        #     track["nuts1"] = ""

        #     return fig

        # elif val in Functions.nuts1_names:

        #     print("This2")

        #     fig = Functions.plot_nuts_country(track["Country"], 2)

        #     return fig

        # else:
        #     fig = go.Figure(figure)

        #     return fig

            print(track)

        else:
            print("THIS3")
            fig = go.Figure(figure)

    else:
        fig = go.Figure(figure)
        
    # Update track data display
    track_data_display = [html.P(f"{key}: {value}") for key, value in track.items() if key != "clicks" or key != ""]

    return fig, track_data_display

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from dash import Dash, html, callback, Output, Input, State, dcc
from dash.dependencies import ALL
import dash
import json

app = Dash(__name__)

# Define the sectors with bolder colors
sectors = [
    {"name": "Technology", "color": "#D32F2F"},
    {"name": "Healthcare", "color": "#1976D2"},
    {"name": "Finance", "color": "#388E3C"},
    {"name": "Energy", "color": "#F57C00"},
    {"name": "Consumer Goods", "color": "#512DA8"},
    {"name": "Utilities", "color": "#00796B"},
    {"name": "Real Estate", "color": "#C2185B"},
    {"name": "Telecommunications", "color": "#689F38"},
    {"name": "Materials", "color": "#0288D1"},
    {"name": "Industrials", "color": "#7B1FA2"},
    {"name": "Consumer Services", "color": "#AFB42B"},
    {"name": "Transportation", "color": "#FBC02D"}
]

# App layout
app.layout = html.Div([
    dcc.Store(id='selected-sectors', data=[]),  # Store to keep track of selected sectors

    # Large Square Container
    html.Div(
        id='sector-container',
        children=[
            # Fancy Smaller squares for each sector
            html.Div(
                sector["name"],
                id={'type': 'sector-box', 'index': i}, 
                n_clicks=0,
                style={
                    'display': 'flex',
                    'justifyContent': 'center',
                    'alignItems': 'center',
                    'border': '2px solid #fff',
                    'borderRadius': '15px',
                    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                    'fontSize': '14px',  # Adjust font size for better text wrapping
                    'cursor': 'pointer',
                    'backgroundColor': sector["color"],
                    'color': '#fff',
                    'textAlign': 'center',
                    'fontWeight': 'bold',
                    'width': '100px',
                    'height': '100px',
                    'padding': '5px',
                    'textOverflow': 'ellipsis',
                    'wordWrap': 'break-word',  # Allow text to wrap onto the next line
                    'overflow': 'hidden',
                    'transition': 'transform 0.2s, box-shadow 0.2s'
                },
                className='sector-box'
            ) for i, sector in enumerate(sectors)
        ],
        style={
            'display': 'grid',
            'gridTemplateColumns': 'repeat(4, 1fr)',
            'gridTemplateRows': 'repeat(3, 1fr)',
            'gap': '10px',
            'justifyItems': 'center',  # Centering small squares horizontally
            'alignItems': 'center',  # Centering small squares vertically
            'width': '500px',
            'height': '370px',
            'border': '2px solid black',
            'borderRadius': '20px',
            'padding': '10px',
            'boxShadow': '0px 10px 15px rgba(0, 0, 0, 0.2)',
            'backgroundColor': '#f9f9f9',
            'margin': '50px auto'
        }
    ),

    # "Okay" Button
    html.Button('Okay', id='okay-button', n_clicks=0, style={
        'display': 'block',
        'margin': '20px auto',
        'padding': '10px 20px',
        'fontSize': '16px',
        'fontWeight': 'bold',
        'color': '#fff',
        'backgroundColor': '#007BFF',
        'border': 'none',
        'borderRadius': '5px',
        'cursor': 'pointer',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    }),

    # Output Box for Displaying Selected Sectors
    html.Div(id='selected-sector', style={
        'width': '460px', 
        'height': 'auto', 
        'border': '2px solid black', 
        'borderRadius': '10px',
        'padding': '10px', 
        'margin': '20px auto',
        'textAlign': 'center',
        'fontSize': '20px',
        'fontWeight': 'bold',
        'backgroundColor': '#fff',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    })
])

# Callback to handle multiple selections
@callback(
    Output('selected-sectors', 'data'),
    Output('sector-container', 'children'),
    Input({'type': 'sector-box', 'index': ALL}, 'n_clicks'),
    State('selected-sectors', 'data')
)
def update_selected_sectors(n_clicks, selected_sectors):
    ctx = dash.callback_context

    # Determine which sector box was clicked
    if not ctx.triggered:
        return selected_sectors, dash.no_update

    clicked_index = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])['index']

    selected_sector = sectors[clicked_index]["name"]

    # Toggle selection
    if selected_sector in selected_sectors:
        selected_sectors.remove(selected_sector)
    else:
        selected_sectors.append(selected_sector)

    # Update the styles of the boxes to reflect selection
    updated_children = [
        html.Div(
            sector["name"],
            id={'type': 'sector-box', 'index': i}, 
            n_clicks=0,
            style={
                'display': 'flex',
                'justifyContent': 'center',
                'alignItems': 'center',
                'border': '2px solid #fff',
                'borderRadius': '15px',
                'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                'fontSize': '14px',  # Adjust font size for better text wrapping
                'cursor': 'pointer',
                'backgroundColor': sector["color"] if sector["name"] not in selected_sectors else '#000000',
                'color': '#fff' if sector["name"] not in selected_sectors else '#00ff00',
                'textAlign': 'center',
                'fontWeight': 'bold',
                'width': '100px',
                'height': '100px',
                'padding': '5px',
                'textOverflow': 'ellipsis',
                'wordWrap': 'break-word',  # Allow text to wrap onto the next line
                'overflow': 'hidden',
                'transition': 'transform 0.2s, box-shadow 0.2s'
            },
            className='sector-box'
        ) for i, sector in enumerate(sectors)
    ]

    return selected_sectors, updated_children

# Callback to handle the "Okay" button click
@callback(
    Output('selected-sector', 'children'),
    Input('okay-button', 'n_clicks'),
    State('selected-sectors', 'data')
)
def handle_okay_click(n_clicks, selected_sectors):
    if n_clicks > 0:
        if not selected_sectors:
            return "No sectors selected"
        return f"Selected Sectors: {', '.join(selected_sectors)}"
    return "Click 'Okay' to confirm your selections"

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from dash import Dash, html, callback, Output, Input, State, dcc
from dash.dependencies import ALL
import json

app = Dash(__name__)

# Define the sectors with bolder colors and their industries
sectors = [
    {"name": "Technology", "color": "#D32F2F", "industries": ["Software", "Hardware", "AI", "Cloud Computing", "Cybersecurity"]},
    {"name": "Healthcare", "color": "#1976D2", "industries": ["Pharmaceuticals", "Medical Devices", "Biotechnology", "Health Insurance", "Telemedicine"]},
    {"name": "Finance", "color": "#388E3C", "industries": ["Banking", "Investment", "Insurance", "Fintech", "Wealth Management"]},
    {"name": "Energy", "color": "#F57C00", "industries": ["Oil & Gas", "Renewable Energy", "Utilities", "Nuclear Energy", "Energy Storage"]},
    {"name": "Consumer Goods", "color": "#512DA8", "industries": ["Food & Beverage", "Household Products", "Apparel", "Personal Care", "Luxury Goods"]},
    {"name": "Utilities", "color": "#00796B", "industries": ["Electricity", "Water", "Gas", "Waste Management", "Telecom Utilities"]},
    {"name": "Real Estate", "color": "#C2185B", "industries": ["Commercial", "Residential", "REITs", "Property Management", "Real Estate Development"]},
    {"name": "Telecommunications", "color": "#689F38", "industries": ["Wireless Services", "Broadband", "Satellite", "Telecom Equipment", "Network Infrastructure"]},
    {"name": "Materials", "color": "#0288D1", "industries": ["Chemicals", "Construction Materials", "Packaging", "Metals & Mining", "Paper & Forest Products"]},
    {"name": "Industrials", "color": "#7B1FA2", "industries": ["Aerospace", "Construction & Engineering", "Machinery", "Transportation Equipment", "Industrial Conglomerates"]},
    {"name": "Consumer Services", "color": "#AFB42B", "industries": ["Retail", "Travel & Leisure", "Education", "Media & Entertainment", "Restaurants"]},
    {"name": "Transportation", "color": "#FBC02D", "industries": ["Airlines", "Railroads", "Shipping", "Logistics", "Trucking"]}
]

# App layout
app.layout = html.Div([
    dcc.Store(id='selected-sectors', data=[]),  # Store to keep track of selected sectors
    dcc.Store(id='selected-industries', data=[]),  # Store to keep track of selected industries

    # Large Square Container for sectors
    html.Div(
        id='sector-container',
        children=[
            # Fancy Smaller squares for each sector
            html.Div(
                sector["name"],
                id={'type': 'sector-box', 'index': i}, 
                n_clicks=0,
                style={
                    'display': 'flex',
                    'justifyContent': 'center',
                    'alignItems': 'center',
                    'border': '2px solid #fff',
                    'borderRadius': '15px',
                    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                    'fontSize': '14px',  # Adjust font size for better text wrapping
                    'cursor': 'pointer',
                    'backgroundColor': sector["color"],
                    'color': '#fff',
                    'textAlign': 'center',
                    'fontWeight': 'bold',
                    'width': '100px',
                    'height': '100px',
                    'padding': '5px',
                    'textOverflow': 'ellipsis',
                    'wordWrap': 'break-word',  # Allow text to wrap onto the next line
                    'overflow': 'hidden',
                    'transition': 'transform 0.2s, box-shadow 0.2s'
                },
                className='sector-box'
            ) for i, sector in enumerate(sectors)
        ],
        style={
            'display': 'grid',
            'gridTemplateColumns': 'repeat(4, 1fr)',
            'gridTemplateRows': 'repeat(3, 1fr)',
            'gap': '10px',
            'justifyItems': 'center',  # Centering small squares horizontally
            'alignItems': 'center',  # Centering small squares vertically
            'width': '460px',
            'height': '370px',
            'border': '2px solid black',
            'borderRadius': '20px',
            'padding': '10px',
            'boxShadow': '0px 10px 15px rgba(0, 0, 0, 0.2)',
            'backgroundColor': '#f9f9f9',
            'margin': '50px auto'
        }
    ),

    # "Okay" Button
    html.Button('Okay', id='okay-button', n_clicks=0, style={
        'display': 'block',
        'margin': '20px auto',
        'padding': '10px 20px',
        'fontSize': '16px',
        'fontWeight': 'bold',
        'color': '#fff',
        'backgroundColor': '#007BFF',
        'border': 'none',
        'borderRadius': '5px',
        'cursor': 'pointer',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    }),

    # Output Box for Displaying Selected Industries
    html.Div(id='industry-container', style={
        'width': '460px', 
        'height': 'auto', 
        'border': '2px solid black', 
        'borderRadius': '10px',
        'padding': '10px', 
        'margin': '20px auto',
        'textAlign': 'center',
        'fontSize': '20px',
        'fontWeight': 'bold',
        'backgroundColor': '#fff',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    })
])

# Callback to handle multiple selections
@callback(
    Output('selected-sectors', 'data'),
    Output('sector-container', 'children'),
    Input({'type': 'sector-box', 'index': ALL}, 'n_clicks'),
    State('selected-sectors', 'data')
)
def update_selected_sectors(n_clicks, selected_sectors):
    ctx = dash.callback_context

    # Determine which sector box was clicked
    if not ctx.triggered:
        return selected_sectors, dash.no_update

    clicked_index = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])['index']

    selected_sector = sectors[clicked_index]["name"]

    # Toggle selection
    if selected_sector in selected_sectors:
        selected_sectors.remove(selected_sector)
    else:
        selected_sectors.append(selected_sector)

    # Update the styles of the boxes to reflect selection
    updated_children = [
        html.Div(
            sector["name"],
            id={'type': 'sector-box', 'index': i}, 
            n_clicks=0,
            style={
                'display': 'flex',
                'justifyContent': 'center',
                'alignItems': 'center',
                'border': '2px solid #fff',
                'borderRadius': '15px',
                'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                'fontSize': '14px',  # Adjust font size for better text wrapping
                'cursor': 'pointer',
                'backgroundColor': sector["color"] if sector["name"] not in selected_sectors else '#000000',
                'color': '#fff' if sector["name"] not in selected_sectors else '#00ff00',
                'textAlign': 'center',
                'fontWeight': 'bold',
                'width': '100px',
                'height': '100px',
                'padding': '5px',
                'textOverflow': 'ellipsis',
                'wordWrap': 'break-word',  # Allow text to wrap onto the next line
                'overflow': 'hidden',
                'transition': 'transform 0.2s, box-shadow 0.2s'
            },
            className='sector-box'
        ) for i, sector in enumerate(sectors)
    ]

    return selected_sectors, updated_children

# Callback to handle the "Okay" button click
@callback(
    Output('industry-container', 'children'),
    Input('okay-button', 'n_clicks'),
    State('selected-sectors', 'data')
)
def display_industries(n_clicks, selected_sectors):
    if n_clicks > 0 and selected_sectors:
        industry_boxes = []
        for sector in sectors:
            if sector["name"] in selected_sectors:
                industry_boxes.extend([
                    html.Div( industry, style={ 'display': 'flex', 
                                               'justifyContent': 'center', 'alignItems': 
                                               'center', 'border': '2px solid #fff', 
                                               'borderRadius': '15px', 'boxShadow': 
                                               '0px 4px 6px rgba(0, 0, 0, 0.1)', 
                                               'fontSize': '14px', 'cursor': 'pointer', 
                                               'backgroundColor': '#555', # Default color for industries 
                                               'color': '#fff', 
                                               'textAlign': 'center', 
                                               'fontWeight': 'bold', 
                                               'width': '100px', 
                                               'height': '100px', 
                                               'padding': '5px', 
                                               'textOverflow': 'ellipsis', 
                                               'wordWrap': 'break-word', 
                                               'overflow': 'hidden', 
                                               'transition': 'transform 0.2s, box-shadow 0.2s', 
                                               'margin': '5px' } ) for industry in sector["industries"] ])
                return html.Div(
            industry_boxes,
            style={
                'display': 'grid',
                'gridTemplateColumns': 'repeat(4, 1fr)',
                'gridAutoRows': 'minmax(100px, auto)',
                'gap': '10px',
                'justifyItems': 'center',
                'alignItems': 'center',
                'width': '100%',
                'margin': '20px 0'
            }
        )
    return "No sectors selected"

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from dash import Dash, html, callback, Output, Input, State, dcc
from dash.dependencies import ALL
import json

app = Dash(__name__)

# Define the sectors with bolder colors and their industries
sectors = [
    {"name": "Technology", "color": "#D32F2F", "industries": ["Software", "Hardware", "AI", "Cloud Computing", "Cybersecurity"]},
    {"name": "Healthcare", "color": "#1976D2", "industries": ["Pharmaceuticals", "Medical Devices", "Biotechnology", "Health Insurance", "Telemedicine"]},
    {"name": "Finance", "color": "#388E3C", "industries": ["Banking", "Investment", "Insurance", "Fintech", "Wealth Management"]},
    {"name": "Energy", "color": "#F57C00", "industries": ["Oil & Gas", "Renewable Energy", "Utilities", "Nuclear Energy", "Energy Storage"]},
    {"name": "Consumer Goods", "color": "#512DA8", "industries": ["Food & Beverage", "Household Products", "Apparel", "Personal Care", "Luxury Goods"]},
    {"name": "Utilities", "color": "#00796B", "industries": ["Electricity", "Water", "Gas", "Waste Management", "Telecom Utilities"]},
    {"name": "Real Estate", "color": "#C2185B", "industries": ["Commercial", "Residential", "REITs", "Property Management", "Real Estate Development"]},
    {"name": "Telecommunications", "color": "#689F38", "industries": ["Wireless Services", "Broadband", "Satellite", "Telecom Equipment", "Network Infrastructure"]},
    {"name": "Materials", "color": "#0288D1", "industries": ["Chemicals", "Construction Materials", "Packaging", "Metals & Mining", "Paper & Forest Products"]},
    {"name": "Industrials", "color": "#7B1FA2", "industries": ["Aerospace", "Construction & Engineering", "Machinery", "Transportation Equipment", "Industrial Conglomerates"]},
    {"name": "Consumer Services", "color": "#AFB42B", "industries": ["Retail", "Travel & Leisure", "Education", "Media & Entertainment", "Restaurants"]},
    {"name": "Transportation", "color": "#FBC02D", "industries": ["Airlines", "Railroads", "Shipping", "Logistics", "Trucking"]}
]

# App layout
app.layout = html.Div([
    dcc.Store(id='selected-sectors', data=[]),  # Store to keep track of selected sectors
    dcc.Store(id='selected-industries', data=[]),  # Store to keep track of selected industries

    # Large Square Container for sectors
    html.Div(
        id='sector-container',
        children=[
            # Fancy Smaller squares for each sector
            html.Div(
                sector["name"],
                id={'type': 'sector-box', 'index': i}, 
                n_clicks=0,
                style={
                    'display': 'flex',
                    'justifyContent': 'center',
                    'alignItems': 'center',
                    'border': '2px solid #fff',
                    'borderRadius': '15px',
                    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                    'fontSize': '14px',  # Adjust font size for better text wrapping
                    'cursor': 'pointer',
                    'backgroundColor': sector["color"],
                    'color': '#fff',
                    'textAlign': 'center',
                    'fontWeight': 'bold',
                    'width': '100px',
                    'height': '100px',
                    'padding': '5px',
                    'textOverflow': 'ellipsis',
                    'wordWrap': 'break-word',  # Allow text to wrap onto the next line
                    'overflow': 'hidden',
                    'transition': 'transform 0.2s, box-shadow 0.2s'
                },
                className='sector-box'
            ) for i, sector in enumerate(sectors)
        ],
        style={
            'display': 'grid',
            'gridTemplateColumns': 'repeat(4, 1fr)',
            'gridTemplateRows': 'repeat(3, 1fr)',
            'gap': '10px',
            'justifyItems': 'center',  # Centering small squares horizontally
            'alignItems': 'center',  # Centering small squares vertically
            'width': '460px',
            'height': '370px',
            'border': '2px solid black',
            'borderRadius': '20px',
            'padding': '10px',
            'boxShadow': '0px 10px 15px rgba(0, 0, 0, 0.2)',
            'backgroundColor': '#f9f9f9',
            'margin': '50px auto'
        }
    ),

    # "Okay" Button
    html.Button('Okay', id='okay-button', n_clicks=0, style={
        'display': 'block',
        'margin': '20px auto',
        'padding': '10px 20px',
        'fontSize': '16px',
        'fontWeight': 'bold',
        'color': '#fff',
        'backgroundColor': '#007BFF',
        'border': 'none',
        'borderRadius': '5px',
        'cursor': 'pointer',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    }),

    # Output Box for Displaying Selected Industries
    html.Div(id='industry-container', style={
        'width': '460px', 
        'height': 'auto', 
        'border': '2px solid black', 
        'borderRadius': '10px',
        'padding': '10px', 
        'margin': '20px auto',
        'textAlign': 'center',
        'fontSize': '20px',
        'fontWeight': 'bold',
        'backgroundColor': '#fff',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    })
])

# Callback to handle multiple selections
@callback(
    Output('selected-sectors', 'data'),
    Output('sector-container', 'children'),
    Input({'type': 'sector-box', 'index': ALL}, 'n_clicks'),
    State('selected-sectors', 'data')
)
def update_selected_sectors(n_clicks, selected_sectors):
    ctx = dash.callback_context

    # Determine which sector box was clicked
    if not ctx.triggered:
        return selected_sectors, dash.no_update

    clicked_index = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])['index']

    selected_sector = sectors[clicked_index]["name"]

    # Toggle selection
    if selected_sector in selected_sectors:
        selected_sectors.remove(selected_sector)
    else:
        selected_sectors.append(selected_sector)

    # Update the styles of the boxes to reflect selection
    updated_children = [
        html.Div(
            sector["name"],
            id={'type': 'sector-box', 'index': i}, 
            n_clicks=0,
            style={
                'display': 'flex',
                'justifyContent': 'center',
                'alignItems': 'center',
                'border': '2px solid #fff',
                'borderRadius': '15px',
                'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                'fontSize': '14px',  # Adjust font size for better text wrapping
                'cursor': 'pointer',
                'backgroundColor': sector["color"] if sector["name"] not in selected_sectors else '#000000',
                'color': '#fff' if sector["name"] not in selected_sectors else '#00ff00',
                'textAlign': 'center',
                'fontWeight': 'bold',
                'width': '100px',
                'height': '100px',
                'padding': '5px',
                'textOverflow': 'ellipsis',
                'wordWrap': 'break-word',  # Allow text to wrap onto the next line
                'overflow': 'hidden',
                'transition': 'transform 0.2s, box-shadow 0.2s'
            },
            className='sector-box'
        ) for i, sector in enumerate(sectors)
    ]

    return selected_sectors, updated_children

# Callback to handle the "Okay" button click
@callback(
    Output('industry-container', 'children'),
    Input('okay-button', 'n_clicks'),
    State('selected-sectors', 'data')
)
def display_industries(n_clicks, selected_sectors):
    if n_clicks > 0 and selected_sectors:
        industry_boxes = []
        for sector in sectors:
            if sector["name"] in selected_sectors:
                industry_boxes.extend([
                    html.Div(
                        industry,
                        style={
                            'display': 'flex',
                            'justifyContent': 'center',
                            'alignItems': 'center',
                            'border': '2px solid #fff',
                            'borderRadius': '15px',
                            'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                            'fontSize': '14px',
                            'cursor': 'pointer',
                            'backgroundColor': '#555',  # Default color for industries
                            'color': '#fff',
                            'textAlign': 'center',
                            'fontWeight': 'bold',
                            'width': '100px',
                            'height': '100px',
                            'padding': '5px',
                            'textOverflow': 'ellipsis',
                            'wordWrap': 'break-word',
                            'overflow': 'hidden',
                            'transition': 'transform 0.2s, box-shadow 0.2s',
                            'margin': '5px'
                        }
                    ) for industry in sector["industries"]
                ])
        
        return html.Div(
            industry_boxes,
            style={
                'display': 'grid',
                'gridTemplateColumns': 'repeat(4, 1fr)',
                'gridAutoRows': 'minmax(100px, auto)',
                'gap': '10px',
                'justifyItems': 'center',
                'alignItems': 'center',
                'width': '100%',
                'margin': '20px 0'
            }
        )
    return "No sectors selected"

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from dash import Dash, html, callback, Output, Input, State, dcc
from dash.dependencies import ALL
import json

app = Dash(__name__)

# Define the sectors with bolder colors
sectors = [
    {"name": "Technology", "color": "#D32F2F"},
    {"name": "Healthcare", "color": "#1976D2"},
    {"name": "Finance", "color": "#388E3C"},
    {"name": "Energy", "color": "#F57C00"},
    {"name": "Consumer Goods", "color": "#512DA8"},
    {"name": "Utilities", "color": "#00796B"},
    {"name": "Real Estate", "color": "#C2185B"},
    {"name": "Telecommunications", "color": "#689F38"},
    {"name": "Materials", "color": "#0288D1"},
    {"name": "Industrials", "color": "#7B1FA2"},
    {"name": "Consumer Services", "color": "#AFB42B"},
    {"name": "Transportation", "color": "#FBC02D"}
]

# Dummy data for industries under each sector
industries = {
    "Technology": ["Software", "Hardware", "AI", "Cybersecurity"],
    "Healthcare": ["Pharmaceuticals", "Medical Devices", "Biotech"],
    "Finance": ["Banking", "Investment", "Insurance"],
    "Energy": ["Oil & Gas", "Renewables", "Nuclear"],
    "Consumer Goods": ["Apparel", "Food & Beverage", "Home Appliances"],
    "Utilities": ["Electricity", "Water", "Gas"],
    "Real Estate": ["Commercial", "Residential", "Industrial"],
    "Telecommunications": ["Mobile", "Broadband", "Satellite"],
    "Materials": ["Chemicals", "Metals", "Paper"],
    "Industrials": ["Aerospace", "Machinery", "Construction"],
    "Consumer Services": ["Retail", "Hospitality", "Entertainment"],
    "Transportation": ["Airlines", "Shipping", "Logistics"]
}

# App layout
app.layout = html.Div([
    dcc.Store(id='selected-sectors', data=[]),  # Store to keep track of selected sectors
    dcc.Store(id='current-screen', data='sectors'),  # Track current screen (sectors or industries)

    # Placeholder for dynamic screen content
    html.Div(id='screen-content'),

    # "Okay" Button
    html.Button('Okay', id='okay-button', n_clicks=0, style={
        'display': 'block',
        'margin': '20px auto',
        'padding': '10px 20px',
        'fontSize': '16px',
        'fontWeight': 'bold',
        'color': '#fff',
        'backgroundColor': '#007BFF',
        'border': 'none',
        'borderRadius': '5px',
        'cursor': 'pointer',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    }),

    # Output Box for Displaying Selected Sectors or Industries
    html.Div(id='selected-sector', style={
        'width': '460px',
        'height': 'auto',
        'border': '2px solid black',
        'borderRadius': '10px',
        'padding': '10px',
        'margin': '20px auto',
        'textAlign': 'center',
        'fontSize': '20px',
        'fontWeight': 'bold',
        'backgroundColor': '#fff',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    })
])

# Callback to update layout on first load
@callback(
    Output('screen-content', 'children'),
    Input('current-screen', 'data'),
    State('selected-sectors', 'data')
)
def display_initial_content(screen, selected_sectors):
    if screen == 'sectors':
        return get_sector_content(selected_sectors)
    elif screen == 'industries':
        return get_industry_content(selected_sectors)
    return dash.no_update

# Combined callback to handle multiple selections and update the screen content
@callback(
    Output('selected-sectors', 'data'),
    Output('screen-content', 'children'),
    Output('selected-sector', 'children'),
    Output('current-screen', 'data'),
    Input({'type': 'sector-box', 'index': ALL}, 'n_clicks'),
    Input('okay-button', 'n_clicks'),
    State('selected-sectors', 'data'),
    State('current-screen', 'data')
)
def update_layout(n_clicks_list, okay_n_clicks, selected_sectors, current_screen):
    ctx = dash.callback_context

    if not ctx.triggered:
        return selected_sectors, dash.no_update, dash.no_update, dash.no_update

    # Handle sector box clicks
    if ctx.triggered[0]['prop_id'].startswith("{\"type\":\"sector-box\""):
        clicked_index = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])['index']
        selected_sector = sectors[clicked_index]["name"]

        # Toggle selection
        if selected_sector in selected_sectors:
            selected_sectors.remove(selected_sector)
        else:
            selected_sectors.append(selected_sector)

    # Handle "Okay" button click
    if 'okay-button.n_clicks' in ctx.triggered[0]['prop_id']:
        if okay_n_clicks > 0:
            if not selected_sectors:
                return selected_sectors, dash.no_update, "No sectors selected", 'sectors'
            return selected_sectors, get_industry_content(selected_sectors), f"Selected Sectors: {', '.join(selected_sectors)}", 'industries'

    # Update sector content (highlighting)
    if current_screen == 'sectors':
        sector_content = get_sector_content(selected_sectors)
    else:
        sector_content = dash.no_update

    return selected_sectors, sector_content, dash.no_update, dash.no_update

def get_sector_content(selected_sectors):
    return html.Div(
        id='sector-container',
        children=[
            html.Div(
                sector["name"],
                id={'type': 'sector-box', 'index': i},
                n_clicks=0,
                style={
                    'display': 'flex',
                    'justifyContent': 'center',
                    'alignItems': 'center',
                    'border': '2px solid #fff',
                    'borderRadius': '15px',
                    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                    'fontSize': '14px',  # Adjust font size for better text wrapping
                    'cursor': 'pointer',
                    'backgroundColor': sector["color"] if sector["name"] not in selected_sectors else '#000000',
                    'color': '#fff' if sector["name"] not in selected_sectors else '#00ff00',
                    'textAlign': 'center',
                    'fontWeight': 'bold',
                    'width': '100px',
                    'height': '100px',
                    'padding': '5px',
                    'textOverflow': 'ellipsis',
                    'wordWrap': 'break-word',  # Allow text to wrap onto the next line
                    'overflow': 'hidden',
                    'transition': 'transform 0.2s, box-shadow 0.2s'
                },
                className='sector-box'
            ) for i, sector in enumerate(sectors)
        ],
        style={
            'display': 'grid',
            'gridTemplateColumns': 'repeat(4, 1fr)',
            'gridTemplateRows': 'repeat(3, 1fr)',
            'gap': '10px',
            'justifyItems': 'center',  # Centering small squares horizontally
            'alignItems': 'center',  # Centering small squares vertically
            'width': '500px',
            'height': '370px',
            'border': '2px solid black',
            'borderRadius': '20px',
            'padding': '10px',
            'boxShadow': '0px 10px 15px rgba(0, 0, 0, 0.2)',
            'backgroundColor': '#f9f9f9',
            'margin': '50px auto'
        }
    )

def get_industry_content(selected_sectors):
    industry_boxes = []
    for sector in selected_sectors:
        sector_industries = industries.get(sector, [])
        for industry in sector_industries:
            industry_boxes.append(
                html.Div(
                    industry,
                    style={
                        'display': 'flex',
                        'justifyContent': 'center',
                        'alignItems': 'center',
                        'border': '2px solid #fff',
                        'borderRadius': '10px',
                        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                        'fontSize': '14px',
                        'cursor': 'pointer',
                        'backgroundColor': '#555',
                        'color': '#fff',
                        'textAlign': 'center',
                        'fontWeight': 'bold',
                        'width': '150px',
                        'height': '60px',
                        'margin': '5px'
                    }
                )
            )
    return html.Div(
        children=industry_boxes,
        style={
            'display': 'flex',
            'flexWrap': 'wrap',
            'justifyContent': 'center',
            'alignItems': 'center',
            'width': '600px',
            'margin': '50px auto',
            'gap': '10px'
        }
    )

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from dash import Dash, html, callback, Output, Input, State, dcc
from dash.dependencies import ALL
import dash
import json

app = Dash(__name__)

# Define the sectors with bolder colors
sectors = [
    {"name": "Technology", "color": "#D32F2F", "industries": ["Software", "Hardware", "IT Services"]},
    {"name": "Healthcare", "color": "#1976D2", "industries": ["Pharmaceuticals", "Medical Devices", "Healthcare Services"]},
    {"name": "Finance", "color": "#388E3C", "industries": ["Banking", "Insurance", "Investment Services"]},
    {"name": "Energy", "color": "#F57C00", "industries": ["Oil & Gas", "Renewable Energy", "Utilities"]},
    {"name": "Consumer Goods", "color": "#512DA8", "industries": ["Food & Beverage", "Household Products", "Apparel"]},
    {"name": "Utilities", "color": "#00796B", "industries": ["Electric", "Water", "Gas"]},
    {"name": "Real Estate", "color": "#C2185B", "industries": ["Residential", "Commercial", "Industrial"]},
    {"name": "Telecommunications", "color": "#689F38", "industries": ["Mobile", "Broadband", "Satellite"]},
    {"name": "Materials", "color": "#0288D1", "industries": ["Chemicals", "Metals & Mining", "Construction Materials"]},
    {"name": "Industrials", "color": "#7B1FA2", "industries": ["Aerospace & Defense", "Machinery", "Industrial Services"]},
    {"name": "Consumer Services", "color": "#AFB42B", "industries": ["Retail", "Travel & Leisure", "Media"]},
    {"name": "Transportation", "color": "#FBC02D", "industries": ["Airlines", "Railroads", "Logistics"]}
]

# Generate the initial sector tiles
initial_sector_tiles = [
    html.Div(
        sector["name"],
        id={'type': 'sector-box', 'index': i}, 
        n_clicks=0,
        style={
            'display': 'flex',
            'justifyContent': 'center',
            'alignItems': 'center',
            'border': '2px solid #fff',
            'borderRadius': '15px',
            'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
            'fontSize': '14px', 
            'cursor': 'pointer',
            'backgroundColor': sector["color"],
            'color': '#fff',
            'textAlign': 'center',
            'fontWeight': 'bold',
            'width': '100px',
            'height': '100px',
            'padding': '5px',
            'textOverflow': 'ellipsis',
            'wordWrap': 'break-word',
            'overflow': 'hidden',
            'transition': 'transform 0.2s, box-shadow 0.2s'
        },
        className='sector-box'
    ) for i, sector in enumerate(sectors)
]

# App layout
app.layout = html.Div([
    dcc.Store(id='selected-sectors', data=[]),  # Store to keep track of selected sectors
    dcc.Store(id='industries-view', data=False),  # Store to track if industries should be displayed

    # Large Square Container
    html.Div(
        id='sector-container',
        children=initial_sector_tiles,  # Set the initial children to the sector tiles
        style={
            'display': 'grid',
            'gridTemplateColumns': 'repeat(4, 1fr)',
            'gridTemplateRows': 'repeat(3, 1fr)',
            'gap': '10px',
            'justifyItems': 'center',
            'alignItems': 'center',
            'width': '500px',
            'height': '370px',
            'border': '2px solid black',
            'borderRadius': '20px',
            'padding': '10px',
            'boxShadow': '0px 10px 15px rgba(0, 0, 0, 0.2)',
            'backgroundColor': '#f9f9f9',
            'margin': '50px auto'
        }
    ),

    # "Okay" Button
    html.Button('Okay', id='okay-button', n_clicks=0, style={
        'display': 'block',
        'margin': '20px auto',
        'padding': '10px 20px',
        'fontSize': '16px',
        'fontWeight': 'bold',
        'color': '#fff',
        'backgroundColor': '#007BFF',
        'border': 'none',
        'borderRadius': '5px',
        'cursor': 'pointer',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    }),

    # Output Box for Displaying Selected Sectors
    html.Div(id='selected-sector', style={
        'width': '460px', 
        'height': 'auto', 
        'border': '2px solid black', 
        'borderRadius': '10px',
        'padding': '10px', 
        'margin': '20px auto',
        'textAlign': 'center',
        'fontSize': '20px',
        'fontWeight': 'bold',
        'backgroundColor': '#fff',
        'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)'
    })
])

# Callback to handle multiple selections and industry display
@callback(
    Output('selected-sectors', 'data'),
    Output('sector-container', 'children'),
    Output('industries-view', 'data'),
    Input({'type': 'sector-box', 'index': ALL}, 'n_clicks'),
    Input('okay-button', 'n_clicks'),
    State('selected-sectors', 'data'),
    State('industries-view', 'data')
)
def update_selected_sectors_or_industries(n_clicks, okay_clicks, selected_sectors, industries_view):
    ctx = dash.callback_context

    if not ctx.triggered:
        return selected_sectors, dash.no_update, dash.no_update

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if 'sector-box' in triggered_id:
        clicked_index = json.loads(triggered_id)['index']
        selected_sector = sectors[clicked_index]["name"]

        # Toggle selection
        if selected_sector in selected_sectors:
            selected_sectors.remove(selected_sector)
        else:
            selected_sectors.append(selected_sector)

        # Update the styles of the boxes to reflect selection
        updated_children = [
            html.Div(
                sector["name"],
                id={'type': 'sector-box', 'index': i}, 
                n_clicks=0,
                style={
                    'display': 'flex',
                    'justifyContent': 'center',
                    'alignItems': 'center',
                    'border': '2px solid #fff',
                    'borderRadius': '15px',
                    'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                    'fontSize': '14px',
                    'cursor': 'pointer',
                    'backgroundColor': sector["color"] if sector["name"] not in selected_sectors else '#000000',
                    'color': '#fff' if sector["name"] not in selected_sectors else '#00ff00',
                    'textAlign': 'center',
                    'fontWeight': 'bold',
                    'width': '100px',
                    'height': '100px',
                    'padding': '5px',
                    'textOverflow': 'ellipsis',
                    'wordWrap': 'break-word',
                    'overflow': 'hidden',
                    'transition': 'transform 0.2s, box-shadow 0.2s'
                },
                className='sector-box'
            ) for i, sector in enumerate(sectors)
        ]

        return selected_sectors, updated_children, industries_view

    elif 'okay-button' in triggered_id and okay_clicks > 0:
        industries_view = True

        # Generate industry tiles
        industry_tiles = []
        for sector in sectors:
            if sector["name"] in selected_sectors:
                for industry in sector["industries"]:
                    industry_tiles.append(
                        html.Div(
                            industry,
                            style={
                                'display': 'flex',
                                'justifyContent': 'center',
                                'alignItems': 'center',
                                'border': '2px solid #fff',
                                'borderRadius': '15px',
                                'boxShadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                                'fontSize': '14px',
                                'backgroundColor': '#333',
                                'color': '#fff',
                                'textAlign': 'center',
                                'fontWeight': 'bold',
                                'width': '100px',
                                'height': '100px',
                                'padding': '5px',
                                'textOverflow': 'ellipsis',
                                'wordWrap': 'break-word',
                                'overflow': 'hidden',
                                'transition': 'transform 0.2s, box-shadow 0.2s'
                            },
                            className='industry-box'
                        )
                    )

        return selected_sectors, industry_tiles, industries_view

    return selected_sectors, dash.no_update, industries_view

# Callback to handle the "Okay" button click
@callback(
    Output('selected-sector', 'children'),
    Input('okay-button', 'n_clicks'),
    State('selected-sectors', 'data')
)
def handle_okay_click(n_clicks, selected_sectors):
    if n_clicks > 0:
        if not selected_sectors:
            return "No sectors selected"
        return f"Selected Sectors: {', '.join(selected_sectors)}"
    return "Click 'Okay' to confirm your selections"

if __name__ == '__main__':
    app.run(debug=True)
